In [1]:
%cd /content
!git clone -b dev1 https://github.com/camenduru/PanoHead

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/ablation-trigridD-1-025000.pkl -d /content/PanoHead/models -o ablation-trigridD-1-025000.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/baseline-easy-khair-025000.pkl -d /content/PanoHead/models -o baseline-easy-khair-025000.pkl
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/PanoHead/resolve/main/easy-khair-180-gpc0.8-trans10-025000.pkl -d /content/PanoHead/models -o easy-khair-180-gpc0.8-trans10-025000.pkl

!pip install imgui glfw pyspng mrcfile ninja plyfile trimesh onnxruntime onnx

%cd /content
!git clone -b dev https://github.com/camenduru/3DDFA_V2
%cd /content/3DDFA_V2
!sh ./build.sh

!cp -rf /content/PanoHead/3DDFA_V2_cropping/test /content/3DDFA_V2
!cp /content/PanoHead/3DDFA_V2_cropping/dlib_kps.py /content/3DDFA_V2
!cp /content/PanoHead/3DDFA_V2_cropping/recrop_images.py /content/3DDFA_V2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/shape_predictor_68_face_landmarks/resolve/main/shape_predictor_68_face_landmarks.dat -d /content/3DDFA_V2 -o shape_predictor_68_face_landmarks.dat

!mkdir /content/in /content/stage /content/output

/content
Cloning into 'PanoHead'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 155 (delta 20), reused 13 (delta 13), pack-reused 126
Receiving objects: 100% (155/155), 29.41 MiB | 12.26 MiB/s, done.
Resolving deltas: 100% (46/46), done.
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 17 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 120901 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.2_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu0.22.04.2) ...
Selecting previously unselected package libaria2-0:amd64.
Preparing to unpa

The input image has to be put in the folder under content/in and the final product will be in the content/output and the image has to be changed to a webp format throgh a real conversion (changing the file extension is for some reason not enough)

In [ ]:
%cd /content/3DDFA_V2
!rm -rf /content/stage/* /content/3DDFA_V2/crop_samples/img/* /content/3DDFA_V2/test/original/* /content/output/*
!cp /content/in/* /content/3DDFA_V2/test/original

!python dlib_kps.py
!python recrop_images.py -i data.pkl -j dataset.json

!cp -rf /content/3DDFA_V2/crop_samples/img/* /content/stage

# %cd /content/PanoHead
# !./gen_pti_script.sh

/content/3DDFA_V2
100% 1/1 [00:01<00:00,  1.87s/it]
results: 1


In [ ]:
%cd /content/PanoHead
!python projector_withseg.py --shapes=True --num-steps=100 --num-steps-pti=100 --outdir=/content/output --target_img=/content/stage --network=/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --idx 0

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
projecting: [0] /content/stage/gab.webp
camera matrix: torch.Size([1, 25])
Computing W midpoint and stddev using 10000 samples...
Setting up PyTorch plugin "bias_act_plugin"... Done.
Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1501: UserWarning: operator() profile_node %106 : int = prim::profile_ivalue(%104)
 does not have profile information (Triggered internally at ../third_party/nvfuser/csrc/graph_fuser.cpp:104.)
  return forward_call(*args, **kwargs)
step:    1/100 mse: 0.30 perc: 0.51 w_norm: 0.00  noise: 0.00 
step:    2/100 mse: 0.37 perc: 0.49 w_norm: 0.00  noise: 0.00 
step:    3/100 mse: 0.29 perc: 0.49 w_norm: 0.00  noise: 0.00 
step:    4/100 mse: 0.16 perc: 0.43 w_norm: 0.00  noise: 0.00 
step:    5/100 mse: 0.10 perc: 0.41 w_norm: 0.00  noise: 0.00 
step:    6/100 mse: 0.11 perc: 0.40 w_

In [ ]:
%cd /content/PanoHead
!python gen_videos_proj_withseg.py --output=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/pre.mp4 --latent=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz --trunc 0.7 --network /content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl --cfg Head

/content/PanoHead
Loading networks from "/content/PanoHead/models/easy-khair-180-gpc0.8-trans10-025000.pkl"...
Traceback (most recent call last):
  File "/content/PanoHead/gen_videos_proj_withseg.py", line 302, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/content/PanoHead/gen_videos_proj_withseg.py", line 296, in generate_images
    ws = torch.tensor(np.load(latent)['w']).to(device)
  File "/usr/local/lib/python3.10/dist-packages/numpy/lib/npyio.py", line 405, in 

In [ ]:
%cd /content/PanoHead
!python gen_videos_proj_withseg.py --output=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/PTI_render/post.mp4 --latent=/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/projected_w.npz --trunc 0.7 --network /content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl --cfg Head

/content/PanoHead
Loading networks from "/content/output/easy-khair-180-gpc0.8-trans10-025000.pkl/0/fintuned_generator.pkl"...
Traceback (most recent call last):
  File "/content/PanoHead/gen_videos_proj_withseg.py", line 301, in <module>
    generate_images() # pylint: disable=no-value-for-parameter
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/usr/local/lib/python3.10/dist-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/content/PanoHead/gen_videos_proj_withseg.py", line 281, in generate_images
    with dnnlib.util.open_url(network_pkl) as f:
  File "/content/PanoHead/dnnlib/util.py", line 405, in open_url
    ret

In [ ]:
!zip -r /content/output.zip /content/output
# first paht is output of zipped file - second path is what to zip compress

In [ ]:
# how to download from a path in the colab files
from google.colab import files
files.download("/content/output.zip")